Track Changes with Thresholds - SR

In [4]:
# Import Modules
import sys
sys.path.append("..") #give this script access to all modules in parent directory
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
import matplotlib.lines as mlines
from scipy.io import loadmat
import scipy
from scipy.stats import skew
import Actigraph_Metrics

# Load Patient Data
# Patients with -2 or 2 are ['Patient13', 'Patient2', 'Patient3', 'Patient4', 'Patient9']
data_dir = r'S:\Sedation_monitoring\PedAccel_directory\PedAccel\data_analysis\PythonPipeline\PatientData'
window_size = 100 # 100 is 1 second worth of time
lead_time = 15
slice_size_min = 16
sr = .5

In [5]:
def fillArrays(signal, Mean, AUC, PEAK_PEAK, Stdev): 
    x = []
    for i in range(len(signal)):
        x.append(i) #x is the x axis for signal data, it is necessary for AUC calculations. Signal is ordered as is x, so this works
    Mean.append(np.mean(signal))
    AUC.append(Actigraph_Metrics.calc_area_under_curve(x,signal))
    PEAK_PEAK.append(max(signal)-min(signal))
    Stdev.append(np.std(signal))

def fill_SBS_Changes(vitals_SBS,SBS_Changes,i):
    state = vitals_SBS[i+1]-vitals_SBS[i]
    if(state > 0):
        SBS_Changes.append(1) #1 signifies increase in SBS
    elif state == 0: 
        SBS_Changes.append(0)
    else:
        SBS_Changes.append(-1)

In [12]:
'''
Iterate through each patient and perform the following:
|_ Start with three class model: {-3, -2; -1, 0; 1, 2}

'''
for patient in os.listdir(data_dir):
    # filter out non-directories
    patient_dir = os.path.join(data_dir, patient)
    print('Processing:', patient)
    if os.path.isdir(patient_dir):        
        data_filepath = os.path.join(patient_dir, f'{patient}_SICKBAY_{lead_time}MIN_{slice_size_min - lead_time}MINNurse.mat')
        
        data = loadmat(data_filepath)
        hr = data['heart_rate']
        SpO2 = data['SpO2']
        rr = data['respiratory_rate']
        x_mag = data['x_mag']
        sbs = data['sbs'].flatten()

        feature_list = [hr,rr,SpO2, x_mag]
        vitals_names = ['hr','rr','spo2', 'x_mag']

        # Create three class SBS
        three_class_sbs = []
        for i in sbs:
            if i == -3 or i == -2:
                three_class_sbs.append(-1)
            if i == -1 or i == 0:
                three_class_sbs.append(0)
            if i == 1 or i == 2:
                three_class_sbs.append(1)

        for k in range(len(feature_list)): 
            cleared_SBS = []
            AUC = []
            PEAK_PEAK = []
            Mean = []
            Stdev = []

            signal_2D = (np.array(feature_list[k])) #signal_2D is a single vitals metric

            for j in range(len(signal_2D)):
                if(np.mean(signal_2D[j])!=0): #check that neither is a flag list, signal_2D[j] is a single window for a single metric
                #define arrays to populate on each iteration
                    # Populate array with changes in SBS
                    cleared_SBS.append(three_class_sbs[j])
                    fillArrays(signal_2D[j], Mean, AUC, PEAK_PEAK, Stdev)
                #else:
                    #print(f'flag list detected at index {j}')

            i = 0
            
            assert(len(Mean)== len(AUC))
            assert(len(AUC) == len(PEAK_PEAK))
            assert(len(cleared_SBS)==len(Mean))
            assert(len(sbs) >= len(cleared_SBS))


            average_change_in_mean_1 = []
            average_change_in_AUC_1 = []
            average_change_in_Peaks_1 = []
            average_change_in_Stdev_1 = []

            average_change_in_mean_minus1 = []
            average_change_in_AUC_minus1 = []
            average_change_in_Peaks_minus1 = []
            average_change_in_Stdev_minus1 = []


            if(len(cleared_SBS) > 2):
                for i in range(len(cleared_SBS)-1):
                    if((cleared_SBS[i] == 0) and cleared_SBS[i+1] == 1):
                        # print(f'The SBS score at index {i} is {cleared_SBS[i]} and the SBS score at index {i + 1} is {cleared_SBS[i+1]}')
                        # print(f'The change in PEAK_PEAK values from {cleared_SBS[i]} to {cleared_SBS[i+1]} is {PEAK_PEAK[i+1]-PEAK_PEAK[i]}')
                        # print(f'The change in AUC from {cleared_SBS[i]} to {cleared_SBS[i+1]} is {AUC[i+1]-AUC[i]}')
                        # print(f'The change in Mean values from {cleared_SBS[i]} to {cleared_SBS[i+1]} is {Mean[i+1]-Mean[i]}')
                        # print(f'The change in Stdev values from {cleared_SBS[i]} to {cleared_SBS[i+1]} is {Stdev[i+1]-Stdev[i]}')

                        average_change_in_mean_1.append(abs(Mean[i+1]-Mean[i]))
                        average_change_in_AUC_1.append(abs(AUC[i+1]-AUC[i]))
                        average_change_in_Peaks_1.append(abs(PEAK_PEAK[i+1]-PEAK_PEAK[i]))
                        average_change_in_Stdev_1.append(abs(Stdev[i+1]-Stdev[i]))

                    elif((cleared_SBS[i] == -1) and (cleared_SBS[i+1] == 0)):
                        average_change_in_mean_minus1.append(abs(Mean[i+1]-Mean[i]))
                        average_change_in_AUC_minus1.append(abs(AUC[i+1]-AUC[i]))
                        average_change_in_Peaks_minus1.append(abs(PEAK_PEAK[i+1]-PEAK_PEAK[i]))
                        average_change_in_Stdev_minus1.append(abs(Stdev[i+1]-Stdev[i]))

                print('_____________________________________________________________________________________________________')
                print(f'******** SUMMARY {vitals_names[k]} {patient}*********\n')
                print(f'The average change in mean from SBS of 0 to 1 was {np.mean(average_change_in_mean_1)}')
                print(f'The average change in AUC from SBS of 0 to 1 was {np.mean(average_change_in_AUC_1)}')
                print(f'The average change in Peak to Peak from SBS 0 to 1 was {np.mean(average_change_in_Peaks_1)}')
                print(f'The average change in Stdev from SBS 0 to 1 was {np.mean(average_change_in_Stdev_1)}\n')

                print(f'The average change in mean from SBS -1 to 0 was {np.mean(average_change_in_mean_minus1)}')
                print(f'The average change in AUC from SBS -1 to 0  was {np.mean(average_change_in_AUC_minus1)}')
                print(f'The average change in Peak to Peak from SBS -1 to 0 was {np.mean(average_change_in_Peaks_minus1)}')
                print(f'The average change in Stdev from SBS -1 to 0 was {np.mean(average_change_in_Stdev_minus1)}')

                print('_____________________________________________________________________________________________________')



Processing: Patient4
_____________________________________________________________________________________________________
******** SUMMARY hr Patient4*********

The average change in mean from SBS of 0 to 1 was 13.439814814814813
The average change in AUC from SBS of 0 to 1 was 6436.925925925926
The average change in Peak to Peak from SBS 0 to 1 was 22.703703703703702
The average change in Stdev from SBS 0 to 1 was 6.93757810647591

The average change in mean from SBS -1 to 0 was nan
The average change in AUC from SBS -1 to 0  was nan
The average change in Peak to Peak from SBS -1 to 0 was nan
The average change in Stdev from SBS -1 to 0 was nan
_____________________________________________________________________________________________________
_____________________________________________________________________________________________________
******** SUMMARY rr Patient4*********

The average change in mean from SBS of 0 to 1 was 5.250094696969697
The average change in AUC from SBS

C:\Users\sraghav9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\sraghav9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


_____________________________________________________________________________________________________
******** SUMMARY x_mag Patient4*********

The average change in mean from SBS of 0 to 1 was 0.0061536082066595554
The average change in AUC from SBS of 0 to 1 was 590.7369478184987
The average change in Peak to Peak from SBS 0 to 1 was 1.0648659467697144
The average change in Stdev from SBS 0 to 1 was 0.012706130743026733

The average change in mean from SBS -1 to 0 was nan
The average change in AUC from SBS -1 to 0  was nan
The average change in Peak to Peak from SBS -1 to 0 was nan
The average change in Stdev from SBS -1 to 0 was nan
_____________________________________________________________________________________________________
Processing: Patient2
_____________________________________________________________________________________________________
******** SUMMARY hr Patient2*********

The average change in mean from SBS of 0 to 1 was 15.376953125000002
The average change in A